In [51]:
%pip install -U sentencepiece
%pip install -U transformers
%pip install -U bitsandbytes
%pip install -U accelerate
%pip install -U huggingface_hub
%pip install -U Biopython
%pip install -U pinecone-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 16.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [62]:
import torch
from transformers import AutoTokenizer, AutoModel
from Bio import Medline
from tqdm import tqdm
from pinecone import Pinecone, PodSpec

In [20]:
if torch.cuda.is_available():
  device = "cuda:0"
else:
  device = "cpu"
device

'cuda:0'

## Load Data
Download pubmed data using linux command line tools:
`esearch -db pubmed -query "intelligence[tiab]" -mindate 2013 -maxdate 2023 | efetch -format medline > ./pubmed_data`

## Preprocessing

In [40]:
records = []
missed = 0

with open("pubmed_data") as stream:
    for article in Medline.parse(stream):

        if not "PMID" in article:
            missed += 1
            continue

        if not "TI" in article:
            missed += 1
            continue

        if not "AB" in article:
            missed += 1
            continue

        article = {
            "id": article["PMID"],
            "metadata": article
        }
         
        records.append(article)

In [19]:
help(Medline)

Help on package Bio.Medline in Bio:

NAME
    Bio.Medline - Code to work with Medline from the NCBI.

DESCRIPTION
    Classes:
     - Record           A dictionary holding Medline data.
    
    Functions:
     - read             Reads one Medline record
     - parse            Allows you to iterate over a bunch of Medline records

PACKAGE CONTENTS


CLASSES
    builtins.dict(builtins.object)
        Record
    
    class Record(builtins.dict)
     |  A dictionary holding information from a Medline record.
     |  
     |  All data are stored under the mnemonic appearing in the Medline
     |  file. These mnemonics have the following interpretations:
     |  
     |  ========= ==============================
     |  Mnemonic  Description
     |  --------- ------------------------------
     |  AB        Abstract
     |  CI        Copyright Information
     |  AD        Affiliation
     |  IRAD      Investigator Affiliation
     |  AID       Article Identifier
     |  AU        Author
  

In [27]:
len(records)

59123

In [28]:
missed

4339

In [29]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2').to(device)

In [30]:
# why not take cls token?
def mean_pooling(last_hidden_state, attention_mask):
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
    return torch.sum(last_hidden_state * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [45]:
batch_size = 64
with torch.no_grad():
    for i in tqdm(range(0, len(records), batch_size)):
        text_samples = [sample["metadata"]["AB"] for sample in records[i:i+batch_size]]

        inputs = tokenizer(text_samples, return_tensors="pt", padding=True, truncation=True).to(device)
        out = model(**inputs)
        pooled = mean_pooling(out.last_hidden_state, inputs["attention_mask"]).to("cpu")


        for i, record in enumerate(records[i:i+batch_size]):
            record["values"] = pooled[i]

100%|██████████| 924/924 [43:45<00:00,  2.84s/it]


In [60]:
pinecone = Pinecone(api_key="7377f10c-e728-4408-b2a6-3ac28daf468f")
index_name = 'pubmed'
pinecone.create_index(
    name=index_name,
    dimension=len(records[0]["values"]),
    metric="cosine",
    spec=PodSpec(environment="gcp-starter") 
)

In [55]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [61]:
batch_size = 32

for i in tqdm(range(0, len(records), batch_size)):
    batch = records[i:i+batch_size]
    batch = [{"id": sample["id"], "values": sample["values"]} for sample in batch]
    index.upsert(batch)

100%|██████████| 1848/1848 [06:37<00:00,  4.65it/s]
